In [0]:
# If notebook is in the repo root, and dashboard is in dashboards/
import os, json, base64, requests
from datetime import datetime

# Get user and context info
user = spark.sql("SELECT current_user()").collect()[0][0]
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
api_url = ctx.apiUrl().get()
token = ctx.apiToken().get()

In [ ]:
# Get current notebook path and compute dashboards folder
notebook_path = ctx.notebookPath().get()
base_path = "/".join(notebook_path.split("/")[:-1])
dashboard_folder = os.path.join(os.getcwd(), "dashboards")
workspace_base_path = f"{base_path}/dashboards"

# Generate timestamp for unique dashboard naming
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
print(f"🕐 Installation timestamp: {timestamp}")

print(f"📂 Dashboard folder path: {dashboard_folder}")
print(f"📁 Found {len(os.listdir(dashboard_folder))} file(s) in {dashboard_folder}")

# Log install info - Comment out the next line to disable telemetry
ENABLE_TELEMETRY = True  # Set to False or comment out to disable usage analytics

if ENABLE_TELEMETRY:
    try:
        # Extract workspace ID from the API URL or notebook path
        workspace_id = api_url.split("/")[2].split(".")[0]  # Gets workspace from URL like https://dbc-abc123.cloud.databricks.com
        
        # Mask the email - only mask first 5 characters
        if '@' in user:
            username, domain = user.split('@')
            if len(username) > 5:
                masked_user = '*' * 5 + username[5:] + '@' + domain
            else:
                masked_user = '*' * len(username) + '@' + domain
        else:
            masked_user = user  # If no @ symbol, use as is
        
        log_response = requests.post(
            'http://87.121.93.91:8080/api/log',
        headers={
            'x-api-key': 'chaplin',
            'Content-Type': 'application/json'
        },
            json={
                'workspace_id': workspace_id,
                'user': masked_user  # Send masked email instead of full email
            },
            timeout=10
        )
        
        if log_response.status_code == 201:
            print(f"📊 Activity logged successfully for workspace: {workspace_id}, user: {masked_user}")
        else:
            print(f"⚠️ Failed to log activity: {log_response.status_code}")
            
    except Exception as e:
        print(f"⚠️ Logging error: {str(e)}")

for fname in os.listdir(dashboard_folder):
    if fname.endswith(".lvdash.json"):
        base_dashboard_name = fname.replace(".lvdash.json", "")
        # Add timestamp to make dashboard name unique
        dashboard_name = f"{base_dashboard_name}_{timestamp}"
        
        print(f"\n📊 Processing: {base_dashboard_name}")
        print(f"   ➡️  Will create as: {dashboard_name}")
        
        with open(os.path.join(dashboard_folder, fname), "r", encoding="utf-8") as f:
            dashboard_def = json.load(f)

        # First, try to create the dashboard
        response = requests.post(
            url=f"{api_url}/api/2.0/lakeview/dashboards",
            headers={
                "Authorization": f"Bearer {token}",
                "Content-Type": "application/json"
            },
            json={
                "display_name": dashboard_name,
                "parent_path": workspace_base_path,
                "serialized_dashboard": json.dumps(dashboard_def),
                "warehouse_id": None  # Will use default warehouse
            }
        )

        if response.status_code == 200 or response.status_code == 201:
            result = response.json()
            dashboard_id = result.get("dashboard_id", "")
            dashboard_path = result.get("path", "")
            print(f"✅ Created Lakeview Dashboard: {dashboard_name}")
            print(f"   📊 Dashboard ID: {dashboard_id}")
            print(f"   🔗 Path: {dashboard_path}")
            print(f"   🌐 URL: {api_url}/sql/dashboardsv3/{dashboard_id}")
        elif response.status_code == 400:
            # Check if it's a RESOURCE_ALREADY_EXISTS error
            try:
                error_json = response.json()
                if error_json.get("error_code") == "RESOURCE_ALREADY_EXISTS":
                    print(f"⚠️  Dashboard '{dashboard_name}' already exists. Finding and updating...")
                    
                    # List all dashboards to find the existing one
                    list_response = requests.get(
                        url=f"{api_url}/api/2.0/lakeview/dashboards",
                        headers={"Authorization": f"Bearer {token}"}
                    )
                    
                    if list_response.status_code == 200:
                        dashboards = list_response.json().get("dashboards", [])
                        existing_dashboard = None
                        
                        # Find the dashboard by name
                        for dash in dashboards:
                            if dash.get("display_name") == dashboard_name or dash.get("display_name") == f"{dashboard_name}.lvdash.json":
                                existing_dashboard = dash
                                break
                        
                        if existing_dashboard:
                            dashboard_id = existing_dashboard["dashboard_id"]
                            
                            # Update the existing dashboard using PATCH
                            update_response = requests.patch(
                                url=f"{api_url}/api/2.0/lakeview/dashboards/{dashboard_id}",
                                headers={
                                    "Authorization": f"Bearer {token}",
                                    "Content-Type": "application/json"
                                },
                                json={
                                    "serialized_dashboard": json.dumps(dashboard_def)
                                }
                            )
                            
                            if update_response.status_code == 200:
                                result = update_response.json()
                                print(f"✅ Updated Lakeview Dashboard: {dashboard_name}")
                                print(f"   📊 Dashboard ID: {dashboard_id}")
                                print(f"   🔗 Path: {existing_dashboard.get('path', '')}")
                                print(f"   🌐 URL: {api_url}/sql/dashboardsv3/{dashboard_id}")
                            else:
                                print(f"❌ Failed to update dashboard: {dashboard_name} — Status: {update_response.status_code}")
                                print(f"Error Response: {update_response.text}")
                        else:
                            print(f"❌ Could not find existing dashboard: {dashboard_name}")
                    else:
                        print(f"❌ Failed to list dashboards — Status: {list_response.status_code}")
                else:
                    print(f"❌ Failed to import: {dashboard_name} — Status: {response.status_code}")
                    print(f"Error Response: {response.text}")
                    print(f"Error Details: {json.dumps(error_json, indent=2)}")
            except:
                print(f"❌ Failed to import: {dashboard_name} — Status: {response.status_code}")
                print(f"Error Response: {response.text}")
        else:
            print(f"❌ Failed to import: {dashboard_name} — Status: {response.status_code}")
            print(f"Error Response: {response.text}")
            try:
                error_json = response.json()
                print(f"Error Details: {json.dumps(error_json, indent=2)}")
            except:
                pass

In [ ]:
# Step 2: Publish Dashboard with Warehouse
print("\n" + "="*70)
print("📤 STEP 2: PUBLISHING DASHBOARD WITH WAREHOUSE")
print("="*70)

# Get available SQL warehouse
warehouses_response = requests.get(
    url=f"{api_url}/api/2.0/sql/warehouses",
    headers={"Authorization": f"Bearer {token}"}
)

warehouse_id = None
if warehouses_response.status_code == 200:
    warehouses = warehouses_response.json().get("warehouses", [])
    if warehouses:
        # Get first running or stopped warehouse
        for wh in warehouses:
            if wh.get("state") in ["RUNNING", "STOPPED"]:
                warehouse_id = wh.get("id")
                warehouse_name = wh.get("name")
                print(f"   🏭 Found warehouse: {warehouse_name} ({warehouse_id})")
                break
        if not warehouse_id:
            # If none running/stopped, take the first one
            warehouse_id = warehouses[0].get("id")
            warehouse_name = warehouses[0].get("name")
            print(f"   🏭 Using warehouse: {warehouse_name} ({warehouse_id})")

if warehouse_id:
    # Publish dashboard with warehouse
    publish_response = requests.post(
        url=f"{api_url}/api/2.0/lakeview/dashboards/{dashboard_id}/publish",
        headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
        json={"warehouse_id": warehouse_id}
    )
    
    if publish_response.status_code in [200, 201]:
        print(f"✅ Dashboard published successfully")
        print(f"   🏭 Warehouse ID: {warehouse_id}")
    else:
        print(f"⚠️  Could not publish via API: {publish_response.status_code}")
        print(f"   You may need to publish manually from the dashboard UI")
else:
    print("⚠️  No warehouse found. Dashboard will be created but not published.")
    print("   You can publish it manually from the dashboard UI")

In [ ]:
# Step 3: Configure Embedding Domains (for Streamlit App)
print("\n" + "="*70)
print("🔐 STEP 3: CONFIGURING EMBEDDING DOMAINS")
print("="*70)
print("   📝 Note: This allows the Streamlit app to embed the dashboard")
print("   📝 (Required for current app functionality, not related to Vector Search)")

embedding_domain = "*.databricksapps.com"
embedding_response = requests.patch(
    url=f"{api_url}/api/2.0/lakeview/dashboards/{dashboard_id}",
    headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
    json={"embedding_allowed_origins": [embedding_domain]}
)

if embedding_response.status_code in [200, 201]:
    print(f"✅ Added embedding domain: {embedding_domain}")
    print(f"   ✅ Streamlit app can now embed the dashboard")
else:
    print(f"⚠️  Could not set via API: {embedding_response.status_code}")
    print(f"📝 Manual step: Open dashboard → Share → Embed dashboard → Add domain: {embedding_domain}")
    print(f"   (This is required for the Streamlit app to display the dashboard)")

In [ ]:
# Step 4: Update App with Dashboard ID
print("\n" + "="*70)
print("🔄 STEP 4: UPDATING APP WITH DASHBOARD ID")
print("="*70)

# Extract workspace ID from API URL
workspace_id = api_url.split("/")[2].split(".")[0].replace("dbc-", "").replace("-", "")

app_path = os.path.join(os.getcwd(), "streamlit-waf-automation", "app.py")

try:
    with open(app_path, "r", encoding="utf-8") as f:
        app_content = f.read()
    
    # Update dashboard ID, instance URL, and workspace ID using regex
    import re
    app_content = re.sub(
        r'DASHBOARD_ID = "[^"]+"',
        f'DASHBOARD_ID = "{dashboard_id}"',
        app_content
    )
    app_content = re.sub(
        r'INSTANCE_URL = "[^"]+"',
        f'INSTANCE_URL = "{api_url}"',
        app_content
    )
    app_content = re.sub(
        r'WORKSPACE_ID = "[^"]+"',
        f'WORKSPACE_ID = "{workspace_id}"',
        app_content
    )
    
    with open(app_path, "w", encoding="utf-8") as f:
        f.write(app_content)
    
    print(f"✅ Updated app.py with dashboard ID")
    print(f"   📍 Dashboard ID: {dashboard_id}")
    print(f"   📍 Instance URL: {api_url}")
    print(f"   📍 Workspace ID: {workspace_id}")
except Exception as e:
    print(f"⚠️  Error updating app.py: {e}")
    print(f"   You may need to update app.py manually with:")
    print(f"   DASHBOARD_ID = \"{dashboard_id}\"")
    print(f"   INSTANCE_URL = \"{api_url}\"")
    print(f"   WORKSPACE_ID = \"{workspace_id}\"")

In [ ]:
# Step 5: Deploy Databricks App
print("\n" + "="*70)
print("🚀 STEP 5: DEPLOYING DATABRICKS APP")
print("="*70)

app_name = "waf-automation-tool"
workspace_path = f"/Users/{user}/waf-app-source"

# Upload app files to workspace using Workspace API
print(f"📤 Uploading app files to {workspace_path}...")

app_source_dir = os.path.join(os.getcwd(), "streamlit-waf-automation")

# Create directory first
mkdir_response = requests.post(
    url=f"{api_url}/api/2.0/workspace/mkdirs",
    headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
    json={"path": workspace_path}
)

# Upload each file
import base64
for item in os.listdir(app_source_dir):
    source = os.path.join(app_source_dir, item)
    if os.path.isfile(source):
        dest_path = f"{workspace_path}/{item}"
        
        # Read file content
        with open(source, "rb") as f:
            content = f.read()
        
        # Upload file
        upload_response = requests.post(
            url=f"{api_url}/api/2.0/workspace/import",
            headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
            json={
                "path": dest_path,
                "content": base64.b64encode(content).decode("utf-8"),
                "format": "AUTO",
                "language": "PYTHON" if item.endswith(".py") else "AUTO",
                "overwrite": True
            }
        )
        
        if upload_response.status_code in [200, 201]:
            print(f"   ✅ Uploaded: {item}")
        else:
            print(f"   ⚠️  Failed to upload {item}: {upload_response.status_code}")

print(f"✅ App files uploaded to {workspace_path}")

# Deploy app using Databricks Apps API
print(f"\n📦 Deploying app: {app_name}")

deploy_response = requests.post(
    url=f"{api_url}/api/2.0/apps/deployments",
    headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
    json={
        "name": app_name,
        "source_code_path": f"/Workspace{workspace_path}"
    }
)

app_url = None
if deploy_response.status_code in [200, 201]:
    result = deploy_response.json()
    deployment_id = result.get("deployment_id", "")
    print(f"✅ App deployed successfully")
    print(f"   🆔 Deployment ID: {deployment_id}")
    
    # Get app URL
    app_info_response = requests.get(
        url=f"{api_url}/api/2.0/apps/{app_name}",
        headers={"Authorization": f"Bearer {token}"}
    )
    
    if app_info_response.status_code == 200:
        app_info = app_info_response.json()
        app_url = app_info.get("url", "")
        if app_url:
            print(f"   🚀 App URL: {app_url}")
else:
    print(f"⚠️  Error deploying app: {deploy_response.status_code}")
    print(f"   Response: {deploy_response.text}")
    print(f"   You may need to deploy manually via Databricks Apps UI")

In [ ]:
# Installation Summary
print("\n" + "="*70)
print("✅ INSTALLATION COMPLETE!")
print("="*70)
print(f"\n📊 Dashboard ID: {dashboard_id}")
print(f"🔗 Dashboard URL: {api_url}/sql/dashboardsv3/{dashboard_id}")
if warehouse_id:
    print(f"🏭 Warehouse ID: {warehouse_id}")
if 'app_url' in locals() and app_url:
    print(f"🚀 App URL: {app_url}")
print(f"\n💡 Next Steps:")
print(f"   1. Open the dashboard URL to verify it's working")
if 'app_url' in locals() and app_url:
    print(f"   2. Open the app URL: {app_url}")
print(f"   3. The dashboard should load with data (warehouse is configured)")
print(f"   4. If dashboard doesn't load in app, manually add *.databricksapps.com to allowed domains:")
print(f"      Dashboard → Share → Embed dashboard → Add domain")
print("\n")